In [129]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from transformers import pipeline

In [130]:
df = pd.read_csv('db/dataProjs.csv', sep=',')
df.head()

,professor,projeto,resumo
0,ADRIANA BORDINI,ExpPC Explorando o Pensamento Computacional pa...,A falta de profissionais bem formados e de mão...
1,ADRIANA BORDINI,ExpPC Explorando o Pensamento Computacional pa...,A falta de profissionais bem formados e de mão...
2,ALAN CARLOS JUNIOR ROSSETTO,Desenvolvimento de metodologias e de ferrament...,Desenvolvimento de ferramentas e metodologias ...
3,ALAN CARLOS JUNIOR ROSSETTO,"Análise, simulação e caracterização de efeitos...",Este projeto detalha as atividades a serem con...
4,ALAN CARLOS JUNIOR ROSSETTO,MIPWise: ‘Inteligência Artificial das Coisas’ ...,Um dos desafios da agricultura é o aprimoramen...


In [131]:
df = df.dropna()

In [132]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1309 entries, 0 to 1432
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   professor  1309 non-null   object
 1   projeto    1309 non-null   object
 2   resumo     1309 non-null   object
dtypes: object(3)
memory usage: 40.9+ KB


In [133]:
x = df['projeto'].value_counts()

In [134]:
# Checagem de valores maiores que 1
valores_maiores_que_1 = x[x > 1]
if not valores_maiores_que_1.empty:
    print("Valores maiores que 1 encontrados:")
    print(valores_maiores_que_1)

Valores maiores que 1 encontrados:
projeto
Programa de Monitoria da UFPEL                                                                                                                                    34
Ciência da Computação (Bacharelado)                                                                                                                               30
Talk Science: divulgação da ciência e da Biotecnologia                                                                                                            22
Simpósio de Biotecnologia: ano VII                                                                                                                                18
Identidade e pertencimento qualificando a formação do biotecnologista: ano V                                                                                      17
                                                                                                                                    

In [135]:
# Checagem de valores nulos
valores_nulos = df['projeto'].isnull().sum()
if valores_nulos > 0:
    print(f"Existem {valores_nulos} valores nulos na coluna 'projeto'.")

## limpeza de valores nulos

## Remoção de Stopwords


In [136]:
nltk.download('stopwords')
stop_words_pt = stopwords.words('portuguese')

[nltk_data] Downloading package stopwords to C:\Users\Polga-
[nltk_data]     Fe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [137]:
new_stop_words = [
    # Abreviações e gírias
    "vc", "tb", "blz", "pq", "n", "q", "ta", "td", "eh", "ok", 
    "dps", "hj", "aq", "cmg", "aki", "tds", "vlw", "obg", "vdd", "mto", "vl",
    "pfv", "qq", "ah", "tá", "tô", "tava", "vcs", "blza", "tpo", 
    
    # Conectores e expressões genéricas
    "então", "aliás", "inclusive", "tipo", "aí", "daí", "logo", 
    "porém", "mas", "assim", "ainda", "sempre", "também", "quando", 
    "onde", "como", "porquê", "porque", "pra", "até", "pois", 
    "mesmo", "já", "só", "aqui", "ali", "lá", "agora", "antes", 
    "depois", "talvez", "quase", "enquanto", "mesmo", "também", "pra", 
    
    # Artigos, pronomes e termos gerais
    "alguma", "nenhuma", "algum", "nenhum", "qualquer", "todo", 
    "todos", "todas", "essa", "esse", "essas", "esses", "aquela", 
    "aquele", "aquelas", "aqueles", "ninguém", "alguém", "tudo", 
    "nada", "meu", "minha", "meus", "minhas", "seu", "sua", "seus", 
    "suas", "dele", "dela", "deles", "delas", "nosso", "nossa", 
    "nossos", "nossas", "qual", "que", "quem", "isso", "isto", 
    "aquilo", "quanto", "quantos", "quantas", 
    
    # Verbos comuns e flexões irrelevantes
    "ser", "estar", "ir", "ter", "haver", "fazer", "ficar", 
    "poder", "ver", "saber", "querer", "dizer", "parecer", 
    "dever", "deixar", "achar", "passar", "andar", "vir", 
    "tornar", "chegar", "começar", "continuar", "usar", "mostrar", 
    "perder", "ganhar", "voltar", "viver", "pensar", "entrar", 
    "encontrar", "escrever", "falar", "perguntar", "responder", 
    
    # Outras palavras genéricas
    "gente", "galera", "pessoal", "coisa", "coisas", "cara", 
    "negócio", "lugar", "vez", "vezes", "tempo", "hora", 
    "minuto", "dia", "semana", "mês", "ano", "forma", 
    "maneira", "modo", "jeito", "caso", "motivo", "razão", 
    "parte", "lado", "fim", "início", "meio", "fundo", 
    "coisas", "fatos", "ideias", "formas", "assuntos", 
    
    # Ruídos técnicos e da web
    "http", "https", "www", "com", "br", "pt", "org", 
    "net", "php", "html", "site", "post", "link", "user", 
    "perfil", "admin", "comentário", "curtir", "responder", 
    "seguir", "repost", "thread", "tema", "tópico", "upload", 
    "download", "file", "fix", "erro", "suporte", "help", 
    "config", "print", "screenshot", "bot", "chat", "msg", 
    
    # Outras
    "kkk", "rs", "zzz", "hahaha", "hue", "huehue", "mano", 
    "véi", "aff", "sdds", "partiu", "curti", "like", "show", 
    "massa", "top", "zika", "véio", "vish", "oxe", "deboa", 
    "miga", "migx", "poxa", "caraca", "parça", "nois", "tamo",
    "to", "golpe", "golpes"
]

In [138]:
stopWords_geral = new_stop_words + stop_words_pt

In [139]:
def clean_SW(text):

    text = re.sub(r'\d+', '', text)
    text = text.lower()
    words = text.split()

    filtragem = [word for word in words if word not in stopWords_geral]

    return ' '.join(filtragem)

In [140]:
df = df[df['resumo'].notna() & (df['resumo'] != '')]


In [141]:
df['resumo_sem_stop_words'] = df['resumo'].apply(clean_SW)

In [142]:
df['resumo_sem_stop_words']

0       falta profissionais bem formados mão obra qual...
1       falta profissionais bem formados mão obra qual...
2       desenvolvimento ferramentas metodologias estim...
3       projeto detalha atividades serem conduzidas de...
4       desafios agricultura aprimoramento sistemas pr...
                              ...                        
1428    bacia hidrográfica mirim-são gonçalo unidade t...
1429    escopo geográfico projeto corresponde toda bac...
1430    criação sistema irrigação chasqueiro resultado...
1431    contexto atual, relacionados ambiente recursos...
1432    crescimento populacional falta fiscalização, p...
Name: resumo_sem_stop_words, Length: 1309, dtype: object

##  Tokenização

In [143]:
text = df['resumo']
df['tokens'] = text.apply(word_tokenize)


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\Polga-Fe/nltk_data'
    - 'c:\\Users\\Polga-Fe\\AppData\\Local\\Programs\\Python\\Python312\\nltk_data'
    - 'c:\\Users\\Polga-Fe\\AppData\\Local\\Programs\\Python\\Python312\\share\\nltk_data'
    - 'c:\\Users\\Polga-Fe\\AppData\\Local\\Programs\\Python\\Python312\\lib\\nltk_data'
    - 'C:\\Users\\Polga-Fe\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:/Users/Polga-Fe/nltk_data'
    - 'C:/Users/Polga-Fe/nltk_data'
    - 'C:/Users/Polga-Fe/nltk_data'
    - 'C:/Users/Polga-Fe/nltk_data'
*************************************************

In [ ]:
# Se quiser aplicar a filtragem de stop words, você pode fazer:
df['filtered_tokens'] = df['tokens'].apply(lambda tokens: [word for word in tokens if word.lower() not in stopWords_geral])

# Exibir as primeiras linhas para verificar
print(df[['resumo', 'tokens', 'filtered_tokens']].head())

KeyError: 'tokens'

In [ ]:
nltk.data.path.append('C:/Users/Polga-Fe/nltk_data')
# Remover valores nulos para evitar problemas
df = df.dropna(subset=['resumo'])

# Converter valores para string apenas por segurança
df['resumo'] = df['resumo'].astype(str)

# Aplicar tokenização de sentenças
df['sentences'] = df['resumo'].apply(nltk.sent_tokenize)


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\Polga-Fe/nltk_data'
    - 'c:\\Users\\Polga-Fe\\AppData\\Local\\Programs\\Python\\Python312\\nltk_data'
    - 'c:\\Users\\Polga-Fe\\AppData\\Local\\Programs\\Python\\Python312\\share\\nltk_data'
    - 'c:\\Users\\Polga-Fe\\AppData\\Local\\Programs\\Python\\Python312\\lib\\nltk_data'
    - 'C:\\Users\\Polga-Fe\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:/Users/Polga-Fe/nltk_data'
    - 'C:/Users/Polga-Fe/nltk_data'
    - 'C:/Users/Polga-Fe/nltk_data'
**********************************************************************


## Normalização


## Stemming

In [144]:
def aplicar_stemming(texto):
    # Inicializa o stemmer
    stemmer = PorterStemmer()

    # Tokeniza o texto
    tokens = nltk.word_tokenize(texto)

    # Aplica stemming para cada token e retorna a lista de palavras stemmadas
    stemmed_tokens = [stemmer.stem(token) for token in tokens]

    # Junta as palavras de volta em uma string
    return ' '.join(stemmed_tokens)

In [145]:
df['resumo_stemmed'] = df['resumo'].apply(aplicar_stemming)


LookupError: 
**********************************************************************
  Resource [93mpunkt_tab[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('punkt_tab')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mtokenizers/punkt_tab/english/[0m

  Searched in:
    - 'C:\\Users\\Polga-Fe/nltk_data'
    - 'c:\\Users\\Polga-Fe\\AppData\\Local\\Programs\\Python\\Python312\\nltk_data'
    - 'c:\\Users\\Polga-Fe\\AppData\\Local\\Programs\\Python\\Python312\\share\\nltk_data'
    - 'c:\\Users\\Polga-Fe\\AppData\\Local\\Programs\\Python\\Python312\\lib\\nltk_data'
    - 'C:\\Users\\Polga-Fe\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:/Users/Polga-Fe/nltk_data'
    - 'C:/Users/Polga-Fe/nltk_data'
    - 'C:/Users/Polga-Fe/nltk_data'
    - 'C:/Users/Polga-Fe/nltk_data'
*************************************************

## Lematização
